In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
!pwd && ls

# Unzip + Import data

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

# Categorize each file so we can create the labels
* first try marked as comment, didn't use because of efficiency problems

In [ ]:
y_datset = []
x_datset = []
#col_names = ['file', 'cat/dog']
#datset = pd.DataFrame(columns=col_names)
min_dim = 0

for file in os.listdir('./train'):
    x_datset.append(file)   
    if file.startswith('cat'):
        y_datset.append('0')
        #datset.loc[len(datset)] = [file, 0]
    else:
        y_datset.append('1')
        #datset.loc[len(datset)] = [file, 1]
print(y_datset[:5])

train_df = pd.DataFrame({'filename': x_datset, 'class': y_datset})
print(train_df.head())

# Create TestSet to evaluate

In [ ]:
from sklearn.model_selection import train_test_split


x_train, x_test= train_test_split(train_df, test_size=0.33, random_state=42)
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

In [ ]:
x_testImg = plt.imread('./train/'+x_test.iloc[0,0])
#print(x_testImg[0])
plt.imshow(x_testImg, cmap=plt.cm.binary)
plt.show()
print(x_test['class'])

In [ ]:
x_trainImg = plt.imread('./train/'+x_train.iloc[0,0])
print(x_trainImg[0])
plt.imshow(x_trainImg, cmap=plt.cm.binary)
plt.show()
print(x_train['class'])

# Preprocessing Data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# create two generators to augment/scale input data
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# create batches of data
dir_train = './train'
x_col = 'filename'
y_col='class'
class_mode = 'binary'
target_size=(224,224)
train_generator = train_datagen.flow_from_dataframe(x_train, directory=dir_train, x_col=x_col, y_col=y_col, class_mode=class_mode, target_size=target_size)
test_generator = test_datagen.flow_from_dataframe(x_test, directory=dir_train, x_col=x_col, y_col=y_col, class_mode=class_mode, target_size=target_size)

In [ ]:
print(type(train_generator))

In [ ]:
img, labels = next(train_generator)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(img)
print(labels)

In [ ]:
img = plt.imread('./train/dog.12493.jpg')
print(img.shape)

# Create Model

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(keras.layers.Conv2D(32, (3, 3), padding='same',input_shape=(224,224,3), activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#eS = keras.callbacks.EarlyStopping(min_delta=1, patience=0.0025)
hist = model.fit_generator(generator=train_generator, validation_data=test_generator, epochs=5, steps_per_epoch=50)

In [ ]:
print(hist.history)

# Prepare + Predict test1.zip

In [ ]:
x_col_test1 = []
for file in os.listdir('./test1'):
    x_col_test1.append(file)
    

dir_test = './test1'
x_test1 = pd.DataFrame({'filename': x_col_test1})
test1_generator = test_datagen.flow_from_dataframe(x_test1, directory=dir_test, x_col=x_col, y_col=None, class_mode=None, target_size=target_size)

In [ ]:
pred = model.predict(test1_generator)

In [ ]:
print(pred.size)

In [ ]:
print(type(pred))
print(np.argmax(pred,1))

subm = pd.DataFrame({'id': np.arange(1,12501), 'label': np.argmax(pred,1)})
subm.to_csv('submission.csv', index=False)
print(subm)